# Коллаборативная фильтрация

Коллаборативная фильтрация основана на идее, что пользователей, похожих на меня, можно использовать для прогнозирования того, насколько мне понравится тот или иной продукт или услуга, которые эти пользователи испытали на себе, а я - нет.   

In [42]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise import accuracy
from surprise import SlopeOne

from surprise.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV

###### reader = Reader()
ratings = pd.read_csv('ratings_small.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
ratings.head()

Буду тестировать разные модели из surprise - библиотека, предназначенная для построения и анализа рекомендательных систем:
* Загружаем датасет.
* Проводим кросс-валидацию. Кросс-валидация — это метод оценки производительности модели на разных подмножествах данных, чтобы оценить насколько хорошо она будет работать на новых данных. Будем использовать 5-кратную кросс-валидацию: данные будут разделены на 5 частей, и модель будет обучена на 4 из них и протестирована на оставшейся 5-й, и это будет повторено 5 раз, чтобы все 5 частей послужили тестовыми.
* Обучаем модель.
* Предсказываем.

## SVD

SVD - это метод линейной алгебры, который используется для разложения матрицы на три другие матрицы. В нашем случае, SVD используется для уменьшения размерности матрицы пользователь-фильм.

Как SVD работает в рекомендательных системах:
1.  Матрица пользователь-фильм: Исходные данные представляются в виде матрицы, где строки — пользователи, столбцы — фильмы, и ячейки содержат известные рейтинги.
2.  Сингулярное разложение: Матрица разлагается на три:
    * Матрица пользователей: Представляет пользователей в векторном пространстве. Каждый ряд соответствует пользователю, а столбцы - факторам, которые могут влиять на предпочтения пользователя (например, фактор "любит драму", "любит комедии").
    * Диагональная матрица сингулярных значений: Содержит сингулярные значения, которые указывают на важность каждого фактора. Значения упорядочены по убыванию.
    * Транспонированная матрица фильмов: Представляет фильмы в векторном пространстве. Каждый столбец соответствует фильму, а строки - факторам, аналогичным факторам из матрицы пользователей.
3.  Уменьшение размерности: Оставляют только самые важные факторы (сингулярные значения) из матрицы сингулярных значений, тем самым сокращая размерность матриц пользователей и фильмов.
4.  Восстановление матрицы: Используя урезанные матрицы, можно восстановить (приблизить) исходную матрицу рейтингов.
5.  Предсказание: Предсказанные рейтинги используются для рекомендации фильмов пользователям.

In [52]:
svd = SVD()

cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()

prediction = svd.predict(1, 302, 3)
print(f"SVD Prediction for user 1 and movie 302: {prediction.est:.2f}")

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9022  0.8958  0.8969  0.8958  0.8918  0.8965  0.0034  
MAE (testset)     0.6964  0.6901  0.6926  0.6884  0.6854  0.6906  0.0037  
Fit time          1.43    1.35    1.04    0.97    0.98    1.15    0.20    
Test time         0.20    0.15    0.11    0.15    0.12    0.15    0.03    
SVD Prediction for user 1 and movie 302: 2.67


## KNNBasic

KNNBasic — это метод, который делает предсказания на основе рейтингов K пользователей, наиболее похожих на текущего пользователя. Он относится к группе алгоритмов "на основе памяти", потому что модель хранит все данные (чаще всего, после преобразования в матрицу схожести) и делает вычисления на основе этих данных, в отличие от алгоритмов "на основе модели", которые строят модель из данных (SVD).

Как KNNBasic работает:

1. Расчет схожести:
  * Вычисляется схожесть между всеми парами пользователей.
  * Используется pearson_baseline (в моем коде). Это модифицированный коэффициент корреляции Пирсона, который учитывает базовые рейтинги пользователей при расчете схожести, что обычно повышает точность. Базовый рейтинг пользователя - это средний рейтинг, который он ставит всем фильмам.
2. Поиск K ближайших соседей:
  * Для текущего пользователя выбираются K пользователей, которые имеют наибольшую схожесть.
3. Предсказание рейтинга:
  * Рейтинг для текущего пользователя и фильма предсказывается на основе рейтингов его K ближайших соседей.

In [53]:
sim_options = {'name': 'pearson_baseline', 'user_based': True}
knn = KNNBasic(sim_options=sim_options)

cross_validate(knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()

prediction = knn.predict(1, 302, 3)
print(f"KNN Prediction for user 1 and movie 302: {prediction.est:.2f}")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0020  0.9928  1.0058  0.9999  0.9867  0.9975  0.0069  
MAE (testset)     0.7707  0.7709  0.7766  0.7713  0.7627  0.7705  0.0045  
Fit time          5.30    6.00    5.28    6.24    9.30    6.43    1.49    
Test time         11.92   12.75   11.17   13.91   16.19 

In [57]:
prediction = knn.predict(1134563454323456754321, 302, 3)
print(f"KNN Prediction for user 1 and movie 302: {prediction.est:.2f}")

KNN Prediction for user 1 and movie 302: 3.54


In [73]:
def get_recommendations_for_new_user(user_id, movie_ids, ratings, new_user_ratings, top_n=30):
    predictions = []

    # Создаем словарь с оценками нового пользователя
    new_user_dict = {row['movieId']: row['rating'] for _, row in new_user_ratings.iterrows()}

    # Получаем средние оценки для каждого фильма от других пользователей
    movie_means = ratings.groupby('movieId')['rating'].mean()

    # Для каждого фильма, который новый пользователь не оценил, вычисляем предсказанную оценку
    for movie_id in movie_ids:
        if movie_id not in new_user_dict:
            similar_users_ratings = ratings[ratings['movieId'] == movie_id]['rating']
            if not similar_users_ratings.empty:
                # вычисляем предсказанное значение как среднее по остальным пользователям
                prediction = movie_means[movie_id] if movie_id in movie_means else 0
                predictions.append((movie_id, prediction))

    # Сортируем предсказания по оценкам
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Возвращаем только top_n рекомендаций
    return predictions[:top_n]

# Получаем уникальные ID фильма из оригинального набора данных
movie_ids = ratings['movieId'].unique()

In [75]:
# Пример использования
new_user_ratings = pd.DataFrame({
    'userId': [99911, 99911, 99911],  # Три записи для нового пользователя
    'movieId': [14, 222, 334],
    'rating': [5, 3, 1]
})

# Рекомендации для нового пользователя
recommended_movies = get_recommendations_for_new_user(99911, movie_ids, ratings, new_user_ratings, top_n=30)
print(recommended_movies)

[(2086, 5.0), (6598, 5.0), (3879, 5.0), (1859, 5.0), (4302, 5.0), (4731, 5.0), (5071, 5.0), (5062, 5.0), (51471, 5.0), (6918, 5.0), (8955, 5.0), (3038, 5.0), (4088, 5.0), (4522, 5.0), (4617, 5.0), (5960, 5.0), (4114, 5.0), (6107, 5.0), (48682, 5.0), (59447, 5.0), (69761, 5.0), (71180, 5.0), (97826, 5.0), (98587, 5.0), (102753, 5.0), (107555, 5.0), (4789, 5.0), (6725, 5.0), (26151, 5.0), (59684, 5.0)]


## SlopeOne

SlopeOne — это алгоритм рекомендательных систем, основанный на сравнении разностей рейтингов между парами предметов (фильмов).

Как работает SlopeOne:

1. Расчет разностей рейтингов:
    * Для каждой пары фильмов вычисляется средняя разность рейтингов, которые поставили им пользователи.
    * Эти разности сохраняются в матрицу разностей.
2. Предсказание рейтинга:
    * Чтобы предсказать рейтинг пользователя для фильма, алгоритм берет рейтинги, которые этот пользователь поставил другим фильмам, и добавляет к ним средние разности между этими фильмами и фильмом, который нужно предсказать.
    * Предсказанный рейтинг является средним из всех этих "скорректированных" рейтингов.
    * Если есть несколько фильмов, которые пользователь оценил, то аналогичные вычисления проводятся для всех этих фильмов, и затем результаты усредняются.

In [36]:
slope_one = SlopeOne()
cross_validate(slope_one, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()

prediction = slope_one.predict(1, 302, 3)
print(f"SlopeOne Prediction for user 1 and movie 302: {prediction.est:.2f}")

Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9229  0.9341  0.9313  0.9316  0.9252  0.9290  0.0042  
MAE (testset)     0.7079  0.7145  0.7128  0.7136  0.7092  0.7116  0.0026  
Fit time          2.46    3.37    2.58    2.64    2.54    2.72    0.33    
Test time         4.74    4.52    4.34    4.52    4.49    4.52    0.13    
SlopeOne Prediction for user 1 and movie 302: 2.54


### Подбор гиперпараметров для SVD

In [23]:
param_grid = {
    'n_factors': [50, 100, 150],
    'reg_all': [0.02, 0.1, 0.5]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)

print("Best RMSE:", gs.best_score['rmse'])
print("Best MAE:", gs.best_score['mae'])
print("Best parameters:", gs.best_params['rmse'])

best_svd_rmse = gs.best_estimator['rmse']

trainset = data.build_full_trainset()
best_svd_rmse.fit(trainset)

prediction = best_svd_rmse.predict(1, 302, 2.5)
print(f"Prediction: {prediction.est:.2f}")

Best RMSE: 0.8910287090640056
Best MAE: 0.687439718534794
Best parameters: {'n_factors': 50, 'reg_all': 0.1}
Prediction: 2.73


* n_factors (количество факторов): Это количество *скрытых факторов*, которые SVD использует для представления пользователей и фильмов.
* reg_all (регуляризация всех параметров): Это параметр, который контролирует *регуляризацию*, которая применяется ко всем параметрам модели.
Регуляризация — это техника, которая помогает предотвратить *переобучение* модели. Она как бы "штрафует" модель за слишком большие или сложные параметры, заставляя её находить более простые и обобщающие решения.
* lr_all (скорость обучения): Это параметр, который контролирует, насколько сильно параметры модели обновляются на каждой итерации обучения. Алгоритм обучения SVD постепенно меняет параметры модели, чтобы сделать предсказания более точными. lr_all определяет, на сколько сильно параметры обновляются на каждой итерации.
* n_epochs (количество эпох): Это количество раз, которое алгоритм SVD проходит через все тренировочные данные в процессе обучения.
Что это такое: Каждая эпоха — это один проход по всему набору данных для обучения модели.

In [32]:
def train_and_evaluate(data, n_iter=10):
    
    param_distributions = {
        'n_factors': randint(20, 200),
        'reg_all': uniform(0.01, 0.5),
        'lr_all': uniform(0.002, 0.02),
        'n_epochs': randint(10, 50)
    }

    rs = RandomizedSearchCV(SVD, param_distributions, measures=['rmse', 'mae'], cv=5, n_iter=n_iter)
    rs.fit(data)

    print("Best RMSE:", rs.best_score['rmse'])
    print("Best MAE:", rs.best_score['mae'])
    print("Best parameters (RMSE):", rs.best_params['rmse'])
    print("Best parameters (MAE):", rs.best_params['mae'])

    best_rmse = rs.best_score['rmse']
    best_mae = rs.best_score['mae']
    best_params_rmse = rs.best_params['rmse']
    best_params_mae = rs.best_params['mae']
    best_svd_rmse = rs.best_estimator['rmse']
    best_svd_mae = rs.best_estimator['mae']

    trainset = data.build_full_trainset()
    best_svd_rmse.fit(trainset)
    best_svd_mae.fit(trainset)

    return {
        'best_rmse': best_rmse,
        'best_mae': best_mae,
        'best_params_rmse': best_params_rmse,
        'best_params_mae': best_params_mae,
        'model_rmse': best_svd_rmse,
        'model_mae': best_svd_mae
    }


num_runs = 5
results = []

# Запускаем обучение и оценку несколько раз
for i in range(num_runs):
    print(f"Run {i+1}/{num_runs}")
    result = train_and_evaluate(data, n_iter=10)
    results.append(result)


Run 1/5
Best RMSE: 0.8845886068824523
Best MAE: 0.6823396790926728
Best parameters (RMSE): {'lr_all': 0.0152356581331288, 'n_epochs': 20, 'n_factors': 120, 'reg_all': 0.13516352219691535}
Best parameters (MAE): {'lr_all': 0.0152356581331288, 'n_epochs': 20, 'n_factors': 120, 'reg_all': 0.13516352219691535}
Run 2/5
Best RMSE: 0.8825978135804469
Best MAE: 0.6791322398987357
Best parameters (RMSE): {'lr_all': 0.014524748165272116, 'n_epochs': 13, 'n_factors': 179, 'reg_all': 0.05750356368218917}
Best parameters (MAE): {'lr_all': 0.014524748165272116, 'n_epochs': 13, 'n_factors': 179, 'reg_all': 0.05750356368218917}
Run 3/5
Best RMSE: 0.8729471580300328
Best MAE: 0.6708102330982632
Best parameters (RMSE): {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}
Best parameters (MAE): {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}
Run 4/5
Best RMSE: 0.8785716626025932
Best MAE: 0.6751491243999539
Bes

In [38]:
# Находим лучшие результаты среди всех запусков
best_rmse_run = min(results, key=lambda x: x['best_rmse'])
best_mae_run = min(results, key=lambda x: x['best_mae'])

# Выводим результаты
print("\nBest Results Overall:")
print(f"Best RMSE: {best_rmse_run['best_rmse']:.4f}")
print(f"Best parameters (RMSE): {best_rmse_run['best_params_rmse']}")
print(f"Best MAE: {best_mae_run['best_mae']:.4f}")
print(f"Best parameters (MAE): {best_mae_run['best_params_mae']}")

# Выводим предсказания
prediction_rmse = best_rmse_run['model_rmse'].predict(1, 302, 2.5)
prediction_mae = best_mae_run['model_mae'].predict(1, 302, 2.5)

print(f"Prediction (Best RMSE model): {prediction_rmse.est:.2f}")
print(f"Prediction (Best MAE model): {prediction_mae.est:.2f}")


Best Results Overall:
Best RMSE: 0.8729
Best parameters (RMSE): {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}
Best MAE: 0.6708
Best parameters (MAE): {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}
Prediction (Best RMSE model): 2.76
Prediction (Best MAE model): 2.83


### Итоговая модель:


In [48]:
reader = Reader()
ratings = pd.read_csv('ratings_small.csv')
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
ratings.head()

best_params_rmse = {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}
best_params_mae = {'lr_all': 0.012953500641312315, 'n_epochs': 44, 'n_factors': 91, 'reg_all': 0.09408867005221243}

# Инициализация SVD с лучшими параметрами (RMSE)
svd = SVD(
    lr_all=best_params_rmse['lr_all'],
    n_epochs=best_params_rmse['n_epochs'],
    n_factors=best_params_rmse['n_factors'],
    reg_all=best_params_rmse['reg_all']
)

cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()

prediction = svd.predict(168559888, 302, 3)
print(f"SVD Prediction for user 1 and movie 302: {prediction.est:.2f}")

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8713  0.8702  0.8728  0.8780  0.8744  0.8734  0.0027  
MAE (testset)     0.6663  0.6691  0.6709  0.6751  0.6721  0.6707  0.0029  
Fit time          15.04   18.08   19.17   17.22   14.69   16.84   1.73    
Test time         1.14    2.02    1.77    1.26    1.17    1.47    0.36    
SVD Prediction for user 1 and movie 302: 3.65


### Топ 30 фильмов для пользователя

In [125]:
def get_top_n_recommendations(model, user_id, data, n=30):
    all_movie_ids = data.df['movieId'].unique()
    rated_movie_ids = data.df[data.df['userId'] == user_id]['movieId'].unique()

    unrated_movie_ids = np.setdiff1d(all_movie_ids, rated_movie_ids)

    predictions = [model.predict(user_id, movie_id) for movie_id in unrated_movie_ids]

    predictions.sort(key=lambda x: x.est, reverse=True)

    top_n_movie_ids = [pred.iid for pred in predictions[:n]]
    return top_n_movie_ids


In [126]:
user_id_to_recommend = 1

top_30_movies = get_top_n_recommendations(best_svd_rmse, user_id_to_recommend, data, n=30)
print(f"Top 30 movies for user {user_id_to_recommend}: {top_30_movies}")

Top 30 movies for user 1: [858, 318, 913, 7502, 969, 926, 1221, 2064, 3462, 5971, 904, 50, 899, 905, 78499, 1252, 106782, 1254, 26729, 1276, 2917, 2019, 1060, 3730, 307, 2300, 1228, 246, 1203, 1945]


# KNN

In [104]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import cross_validate

# Настройки для KNN
sim_options = {'name': 'pearson_baseline', 'user_based': True}
knn = KNNBasic(sim_options=sim_options)

# Кросс-валидация
cross_validate(knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Обучаем модель на всех данных
trainset = data.build_full_trainset()
knn.fit(trainset)

# Функция для предсказания для нового пользователя
def predict_for_new_user(new_user_id, user_ratings, movie_id_to_predict):
    # Создаем временный DataFrame для нового пользователя
    new_user_df = pd.DataFrame(user_ratings, columns=['userId', 'movieId', 'rating'])

    # Объединяем старые и новые данные
    ratings_with_new_user = pd.concat([ratings, new_user_df], ignore_index=True)

    # Создаем новый набор данных
    new_data = Dataset.load_from_df(ratings_with_new_user[['userId', 'movieId', 'rating']], reader)

    # Обучаем модель на обновленных данных
    new_trainset = new_data.build_full_trainset()
    knn.fit(new_trainset)

    # Предсказание для нового пользователя и нового фильма
    prediction = knn.predict(new_user_id, movie_id_to_predict)

    return prediction

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9990  0.9881  1.0066  0.9976  1.0006  0.9984  0.0060  
MAE (testset)     0.7720  0.7641  0.7776  0.7692  0.7711  0.7708  0.0043  
Fit time          6.20    5.43    5.82    5.82    5.84    5.83    0.24    
Test time         13.07   11.83   12.86   12.19   13.50 

In [105]:
# Пример новых оценок для нового пользователя
new_user_ratings = [
    (2111, 14, 1),
    (2111, 222, 2),
    (2111, 334, 5)
]

# Предсказание для нового пользователя о фильме 302
movie_id_to_predict = 302
prediction = predict_for_new_user(2111, new_user_ratings, movie_id_to_predict)
print(f"KNN Prediction for user {prediction.uid} and movie {prediction.iid}: {prediction.est:.2f}")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
KNN Prediction for user 2111 and movie 302: 3.00


In [106]:
# Пример новых оценок для нового пользователя
new_user_ratings = [
    (2111, 14, 1),
    (2111, 222, 2),
    (2111, 334, 5)
]

# Предсказание для нового пользователя о фильме 302
movie_id_to_predict = 31
prediction = predict_for_new_user(2111, new_user_ratings, movie_id_to_predict)
print(f"KNN Prediction for user {prediction.uid} and movie {prediction.iid}: {prediction.est:.2f}")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
KNN Prediction for user 2111 and movie 31: 3.00


In [108]:
# Пример новых оценок для нового пользователя
new_user_ratings = [
    (99911, 14, 5),
    (99911, 222, 4),
    (99911, 334, 1)
]

# Предсказание для нового пользователя о фильме 302
movie_id_to_predict = 31
prediction = predict_for_new_user(99911, new_user_ratings, movie_id_to_predict)
print(f"KNN Prediction for user {prediction.uid} and movie {prediction.iid}: {prediction.est:.2f}")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
KNN Prediction for user 99911 and movie 31: 2.50


In [ ]:
# Пример новых оценок для нового пользователя
new_user_ratings = [
    (22111, 31, 1),
    (22111, 1029, 4),
    (22111, 1061, 5)
]

# Предсказание для нового пользователя о фильме 302
movie_id_to_predict = 31
prediction = predict_for_new_user(2111, new_user_ratings, movie_id_to_predict)
print(f"KNN Prediction for user {prediction.uid} and movie {prediction.iid}: {prediction.est:.2f}")

In [110]:
# Пример новых оценок для нового пользователя
new_user_ratings = [
    (221113, 31, 5),
    (221113, 1029, 4),
    (221113, 1129, 5)
]
# Предсказание для нового пользователя
movie_id_to_predict = 302
prediction = predict_for_new_user(221113, new_user_ratings, movie_id_to_predict)
print(f"KNN Prediction for user {prediction.uid} and movie {prediction.iid}: {prediction.est:.2f}")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
KNN Prediction for user 221113 and movie 302: 4.00


In [112]:
# Пример новых оценок для нового пользователя
new_user_ratings = [
    (221113, 1339, 1),
    (221113, 1343, 4),
    (221113, 235, 1)
]
# Предсказание для нового пользователя
movie_id_to_predict = 302
prediction = predict_for_new_user(221113, new_user_ratings, movie_id_to_predict)
print(f"KNN Prediction for user {prediction.uid} and movie {prediction.iid}: {prediction.est:.2f}")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
KNN Prediction for user 221113 and movie 302: 3.00
